# Variants filtering

### 0. import data

In [1]:
import hail as hl

In [2]:
mt= hl.import_vcf('/titan2/UDP_SV/temp_single_hom_5.vcf', reference_genome='GRCh38')

Initializing Hail with default parameters...
Running on Apache Spark version 2.4.1
SparkUI available at http://163.152.180.157:4043
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.49-11ae8408bad0
LOGGING: writing to /home/titan/Hail/UDP/hail-20200717-1724-0.2.49-11ae8408bad0.log


In [3]:
import gzip
with open('/titan2/UDP_SV/temp_single_hom_5.vcf', 'rt') as f:
    for l in f:
        if 'ID=CSQ' in l:
            temp = l.strip('\n').split('|')
            break

temp[0] = 'Allele';temp[72] = 'gnomADg_AF'

print(temp)
print(len(temp))
#print(temp.index('Consequence'))

['Allele', 'Consequence', 'IMPACT', 'SYMBOL', 'Gene', 'Feature_type', 'Feature', 'BIOTYPE', 'EXON', 'INTRON', 'HGVSc', 'HGVSp', 'cDNA_position', 'CDS_position', 'Protein_position', 'Amino_acids', 'Codons', 'Existing_variation', 'DISTANCE', 'STRAND', 'FLAGS', 'VARIANT_CLASS', 'SYMBOL_SOURCE', 'HGNC_ID', 'CANONICAL', 'MANE', 'TSL', 'APPRIS', 'CCDS', 'ENSP', 'SWISSPROT', 'TREMBL', 'UNIPARC', 'SOURCE', 'GENE_PHENO', 'NEAREST', 'SIFT', 'PolyPhen', 'DOMAINS', 'miRNA', 'HGVS_OFFSET', 'AF', 'AFR_AF', 'AMR_AF', 'EAS_AF', 'EUR_AF', 'SAS_AF', 'AA_AF', 'EA_AF', 'gnomAD_AF', 'gnomAD_AFR_AF', 'gnomAD_AMR_AF', 'gnomAD_ASJ_AF', 'gnomAD_EAS_AF', 'gnomAD_FIN_AF', 'gnomAD_NFE_AF', 'gnomAD_OTH_AF', 'gnomAD_SAS_AF', 'MAX_AF', 'MAX_AF_POPS', 'CLIN_SIG', 'SOMATIC', 'PHENO', 'PUBMED', 'MOTIF_NAME', 'MOTIF_POS', 'HIGH_INF_POS', 'MOTIF_SCORE_CHANGE', 'SV_overlap_AF', 'SV_overlap_PC', 'SV_overlap_name', 'gnomADg', 'gnomADg_AF']
73


### 1. Variant filtering
> - `filter == PASS`인 variant만 남기기
> - multi allelic 확인 및 제외
> - LCR(low complexity region) 제외
> - gnomad filtering
> - ~AC ==1/2(hom)~

In [4]:
##  `filter == PASS`인 variant만 남기기 
mt = mt.filter_rows(hl.len(mt.filters) == 0)

## multi allelic 제외
mt = hl.split_multi(mt)
mt = mt.filter_rows(mt.was_split == False)

## gnomad filtering
mt = mt.annotate_rows(csq = mt.info.CSQ)
mt = mt.transmute_rows(csq_gnomADg_AF = mt.csq.map(lambda x: x.split('\|')[72]))
mt = mt.transmute_rows(csq_gnomADg_AF = mt.csq_gnomADg_AF.map(lambda x:hl.cond(x=='',0,hl.float(x))))
mt = mt.filter_rows(mt.csq_gnomADg_AF.all(lambda x: hl.float(x) < 0.001))

In [5]:
mt.count()

2020-07-17 17:25:09 Hail: INFO: Coerced sorted dataset
2020-07-17 17:25:09 Hail: INFO: Coerced sorted dataset


(69, 1)

---

In [7]:
## fam, role annotation
mt = mt.annotate_cols(fam = mt.s.split('\-')[0], role = mt.s.split('\-')[1])

---

# UDP Proband

In [8]:
mt_proband = mt.filter_cols(~((mt.s =='wgs_3-2') | (mt.s =='wgs_3-3') | (mt.s =='wgs_8-2') | (mt.s =='wgs_8-3')))

In [9]:
mt_proband.write('temp_proband_hom.mt', overwrite=True)

2020-07-13 16:51:27 Hail: INFO: Ordering unsorted dataset with network shuffle
2020-07-13 16:51:28 Hail: INFO: Ordering unsorted dataset with network shuffle
2020-07-13 16:51:28 Hail: INFO: Coerced sorted dataset
2020-07-13 16:51:29 Hail: INFO: wrote matrix table with 306 rows and 5 columns in 2 partitions to temp_proband_hom.mt


In [10]:
mt_proband = hl.read_matrix_table('temp_proband_hom.mt')

In [11]:
mt_proband.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
    'fam': str
    'role': str
----------------------------------------
Row fields:
    'locus': locus<GRCh38>
    'alleles': array<str>
    'rsid': str
    'qual': float64
    'filters': set<str>
    'info': struct {
        CIEND: array<str>, 
        CIPOS: array<str>, 
        CHR2: str, 
        END: int32, 
        MAPQ: int32, 
        RE: int32, 
        IMPRECISE: bool, 
        PRECISE: bool, 
        SVLEN: int32, 
        SVMETHOD: str, 
        SVTYPE: str, 
        SUPP_VEC: str, 
        SUPP: str, 
        STRANDS: str, 
        CSQ: array<str>, 
        gnomADg: array<str>, 
        gnomADg_AF: array<str>
    }
    'a_index': int32
    'was_split': bool
    'old_locus': locus<GRCh38>
    'old_alleles': array<str>
    'csq_gnomADg_AF': array<float64>
----------------------------------------
Entry fields:
    'GT': call
    'PSV': str
    

In [6]:
tb = mt.entries()
tb = tb.key_by(tb.locus,tb.alleles,tb.s)

2020-07-17 17:25:28 Hail: WARN: entries(): Resulting entries table is sorted by '(row_key, col_key)'.
    To preserve row-major matrix table order, first unkey columns with 'key_cols_by()'


### 3. HQ het filtering

In [9]:
##필터링 예시 남겨둠
mt_snv_het_HQ = mt_snv_p.filter_rows((mt_snv_p.info.QD>=3) &
                                   (mt_snv_p.info.SOR<=2.5) &
                                   (mt_snv_p.info.ReadPosRankSum>=-1.4) &
                                   (mt_snv_p.variant_qc.gq_stats.mean>=50)&
                                   (mt_snv_p.info.MQRankSum>=-1.7))
mt_snv_het_HQ = mt_snv_het_HQ.annotate_entries(AB=hl.cond(mt_snv_het_HQ.GT.is_het(),
                                                          hl.min(mt_snv_het_HQ.AD.map(lambda x: x/mt_snv_het_HQ.DP)),0))
mt_snv_het_HQ = mt_snv_het_HQ.filter_entries((mt_snv_het_HQ.GT.is_het())&
                                             (mt_snv_het_HQ.GQ>=99)&
                                             (mt_snv_het_HQ.DP>=10)&
                                             (mt_snv_het_HQ.AB>=0.24)&
                                             (mt_snv_het_HQ.AB<=0.76))
mt_snv_het_HQ = hl.sample_qc(mt_snv_het_HQ)

---

### 4. Export data + CSQ annotation


(CSQ 전체)

In [7]:
def CSQ(table):
    t = table
    t = t.annotate(v1 = hl.tuple([t.locus.contig.replace("chr", ""),hl.str(t.locus.position)]),
                   v2 = hl.tuple([t.alleles[0],t.alleles[1]]))
    t = t.transmute(variant = hl.delimit(hl.array([t.v1[0],t.v1[1],t.v2[0],t.v2[1]]), ":"))
    t = t.annotate(CSQ= t.info.CSQ).explode('CSQ')
    t = t.transmute(csq = t.CSQ.split('\|')).explode('csq')
    return t

In [8]:
tb = CSQ(tb)

In [9]:
tb.export("/titan2/UDP_SV/df_csq_single_hom_5.tsv")

2020-07-17 17:25:33 Hail: INFO: Coerced sorted dataset
2020-07-17 17:25:34 Hail: INFO: Coerced sorted dataset
2020-07-17 17:25:34 Hail: INFO: Coerced sorted dataset
2020-07-17 17:25:42 Hail: INFO: merging 2 files totalling 856.3M...
2020-07-17 17:25:43 Hail: INFO: while writing:
    /titan2/UDP_SV/df_csq_single_hom_5.tsv
  merge time: 1.283s


In [10]:
import hail as hl

In [11]:
mt= hl.import_vcf('/titan2/UDP_SV/temp_single_hom_9.vcf', reference_genome='GRCh38')

In [12]:
import gzip
with open('/titan2/UDP_SV/temp_single_hom_9.vcf', 'rt') as f:
    for l in f:
        if 'ID=CSQ' in l:
            temp = l.strip('\n').split('|')
            break

temp[0] = 'Allele';temp[72] = 'gnomADg_AF'

print(temp)
print(len(temp))
#print(temp.index('Consequence'))

['Allele', 'Consequence', 'IMPACT', 'SYMBOL', 'Gene', 'Feature_type', 'Feature', 'BIOTYPE', 'EXON', 'INTRON', 'HGVSc', 'HGVSp', 'cDNA_position', 'CDS_position', 'Protein_position', 'Amino_acids', 'Codons', 'Existing_variation', 'DISTANCE', 'STRAND', 'FLAGS', 'VARIANT_CLASS', 'SYMBOL_SOURCE', 'HGNC_ID', 'CANONICAL', 'MANE', 'TSL', 'APPRIS', 'CCDS', 'ENSP', 'SWISSPROT', 'TREMBL', 'UNIPARC', 'SOURCE', 'GENE_PHENO', 'NEAREST', 'SIFT', 'PolyPhen', 'DOMAINS', 'miRNA', 'HGVS_OFFSET', 'AF', 'AFR_AF', 'AMR_AF', 'EAS_AF', 'EUR_AF', 'SAS_AF', 'AA_AF', 'EA_AF', 'gnomAD_AF', 'gnomAD_AFR_AF', 'gnomAD_AMR_AF', 'gnomAD_ASJ_AF', 'gnomAD_EAS_AF', 'gnomAD_FIN_AF', 'gnomAD_NFE_AF', 'gnomAD_OTH_AF', 'gnomAD_SAS_AF', 'MAX_AF', 'MAX_AF_POPS', 'CLIN_SIG', 'SOMATIC', 'PHENO', 'PUBMED', 'MOTIF_NAME', 'MOTIF_POS', 'HIGH_INF_POS', 'MOTIF_SCORE_CHANGE', 'SV_overlap_AF', 'SV_overlap_PC', 'SV_overlap_name', 'gnomADg', 'gnomADg_AF']
73


### 1. Variant filtering
> - `filter == PASS`인 variant만 남기기
> - multi allelic 확인 및 제외
> - LCR(low complexity region) 제외
> - gnomad filtering
> - ~AC ==1/2(hom)~

In [13]:
##  `filter == PASS`인 variant만 남기기 
mt = mt.filter_rows(hl.len(mt.filters) == 0)

## multi allelic 제외
mt = hl.split_multi(mt)
mt = mt.filter_rows(mt.was_split == False)


## lcr(low complexity region)제외
#lcr_bed = hl.import_bed('../../Resources/lcr/LCR-hs38.bed', reference_genome = 'GRCh38')
#mt = mt.filter_rows(~hl.is_defined(lcr_bed[mt.locus]))

## gnomad filtering
mt = mt.annotate_rows(csq = mt.info.CSQ)
mt = mt.transmute_rows(csq_gnomADg_AF = mt.csq.map(lambda x: x.split('\|')[72]))
mt = mt.transmute_rows(csq_gnomADg_AF = mt.csq_gnomADg_AF.map(lambda x:hl.cond(x=='',0,hl.float(x))))
mt = mt.filter_rows(mt.csq_gnomADg_AF.all(lambda x: hl.float(x) < 0.001))

In [14]:
mt.count()

2020-07-17 17:26:22 Hail: INFO: Coerced sorted dataset
2020-07-17 17:26:23 Hail: INFO: Coerced sorted dataset


(62, 1)

---

In [15]:
tb = mt.entries()
tb = tb.key_by(tb.locus,tb.alleles,tb.s)

### 3. HQ het filtering

In [9]:
##필터링 예시 남겨둠
mt_snv_het_HQ = mt_snv_p.filter_rows((mt_snv_p.info.QD>=3) &
                                   (mt_snv_p.info.SOR<=2.5) &
                                   (mt_snv_p.info.ReadPosRankSum>=-1.4) &
                                   (mt_snv_p.variant_qc.gq_stats.mean>=50)&
                                   (mt_snv_p.info.MQRankSum>=-1.7))
mt_snv_het_HQ = mt_snv_het_HQ.annotate_entries(AB=hl.cond(mt_snv_het_HQ.GT.is_het(),
                                                          hl.min(mt_snv_het_HQ.AD.map(lambda x: x/mt_snv_het_HQ.DP)),0))
mt_snv_het_HQ = mt_snv_het_HQ.filter_entries((mt_snv_het_HQ.GT.is_het())&
                                             (mt_snv_het_HQ.GQ>=99)&
                                             (mt_snv_het_HQ.DP>=10)&
                                             (mt_snv_het_HQ.AB>=0.24)&
                                             (mt_snv_het_HQ.AB<=0.76))
mt_snv_het_HQ = hl.sample_qc(mt_snv_het_HQ)

---

### 4. Export data + CSQ annotation


(CSQ 전체)

In [16]:
def CSQ(table):
    t = table
    t = t.annotate(v1 = hl.tuple([t.locus.contig.replace("chr", ""),hl.str(t.locus.position)]),
                   v2 = hl.tuple([t.alleles[0],t.alleles[1]]))
    t = t.transmute(variant = hl.delimit(hl.array([t.v1[0],t.v1[1],t.v2[0],t.v2[1]]), ":"))
    t = t.annotate(CSQ= t.info.CSQ).explode('CSQ')
    t = t.transmute(csq = t.CSQ.split('\|')).explode('csq')
    return t

In [17]:
tb = CSQ(tb)

In [18]:
tb.export("/titan2/UDP_SV/df_csq_single_hom_9.tsv")

2020-07-17 17:26:30 Hail: INFO: Coerced sorted dataset
2020-07-17 17:26:30 Hail: INFO: Coerced sorted dataset
2020-07-17 17:26:30 Hail: INFO: Coerced sorted dataset
2020-07-17 17:26:36 Hail: INFO: merging 2 files totalling 717.3M...
2020-07-17 17:26:37 Hail: INFO: while writing:
    /titan2/UDP_SV/df_csq_single_hom_9.tsv
  merge time: 1.170s


In [19]:
import hail as hl

In [20]:
mt= hl.import_vcf('/titan2/UDP_SV/temp_single_hom_12.vcf', reference_genome='GRCh38')

In [21]:
import gzip
with open('/titan2/UDP_SV/temp_single_hom_12.vcf', 'rt') as f:
    for l in f:
        if 'ID=CSQ' in l:
            temp = l.strip('\n').split('|')
            break

temp[0] = 'Allele';temp[72] = 'gnomADg_AF'

print(temp)
print(len(temp))
#print(temp.index('Consequence'))

['Allele', 'Consequence', 'IMPACT', 'SYMBOL', 'Gene', 'Feature_type', 'Feature', 'BIOTYPE', 'EXON', 'INTRON', 'HGVSc', 'HGVSp', 'cDNA_position', 'CDS_position', 'Protein_position', 'Amino_acids', 'Codons', 'Existing_variation', 'DISTANCE', 'STRAND', 'FLAGS', 'VARIANT_CLASS', 'SYMBOL_SOURCE', 'HGNC_ID', 'CANONICAL', 'MANE', 'TSL', 'APPRIS', 'CCDS', 'ENSP', 'SWISSPROT', 'TREMBL', 'UNIPARC', 'SOURCE', 'GENE_PHENO', 'NEAREST', 'SIFT', 'PolyPhen', 'DOMAINS', 'miRNA', 'HGVS_OFFSET', 'AF', 'AFR_AF', 'AMR_AF', 'EAS_AF', 'EUR_AF', 'SAS_AF', 'AA_AF', 'EA_AF', 'gnomAD_AF', 'gnomAD_AFR_AF', 'gnomAD_AMR_AF', 'gnomAD_ASJ_AF', 'gnomAD_EAS_AF', 'gnomAD_FIN_AF', 'gnomAD_NFE_AF', 'gnomAD_OTH_AF', 'gnomAD_SAS_AF', 'MAX_AF', 'MAX_AF_POPS', 'CLIN_SIG', 'SOMATIC', 'PHENO', 'PUBMED', 'MOTIF_NAME', 'MOTIF_POS', 'HIGH_INF_POS', 'MOTIF_SCORE_CHANGE', 'SV_overlap_AF', 'SV_overlap_PC', 'SV_overlap_name', 'gnomADg', 'gnomADg_AF']
73


### 1. Variant filtering
> - `filter == PASS`인 variant만 남기기
> - multi allelic 확인 및 제외
> - LCR(low complexity region) 제외
> - gnomad filtering
> - ~AC ==1/2(hom)~

In [22]:
##  `filter == PASS`인 variant만 남기기 
mt = mt.filter_rows(hl.len(mt.filters) == 0)

## multi allelic 제외
mt = hl.split_multi(mt)
mt = mt.filter_rows(mt.was_split == False)


## lcr(low complexity region)제외
#lcr_bed = hl.import_bed('../../Resources/lcr/LCR-hs38.bed', reference_genome = 'GRCh38')
#mt = mt.filter_rows(~hl.is_defined(lcr_bed[mt.locus]))

## gnomad filtering
mt = mt.annotate_rows(csq = mt.info.CSQ)
mt = mt.transmute_rows(csq_gnomADg_AF = mt.csq.map(lambda x: x.split('\|')[72]))
mt = mt.transmute_rows(csq_gnomADg_AF = mt.csq_gnomADg_AF.map(lambda x:hl.cond(x=='',0,hl.float(x))))
mt = mt.filter_rows(mt.csq_gnomADg_AF.all(lambda x: hl.float(x) < 0.001))

In [23]:
mt.count()

2020-07-17 17:26:45 Hail: INFO: Coerced sorted dataset
2020-07-17 17:26:45 Hail: INFO: Coerced sorted dataset


(65, 1)

---

In [24]:
tb = mt.entries()
tb = tb.key_by(tb.locus,tb.alleles,tb.s)

### 3. HQ het filtering

In [9]:
##필터링 예시 남겨둠
mt_snv_het_HQ = mt_snv_p.filter_rows((mt_snv_p.info.QD>=3) &
                                   (mt_snv_p.info.SOR<=2.5) &
                                   (mt_snv_p.info.ReadPosRankSum>=-1.4) &
                                   (mt_snv_p.variant_qc.gq_stats.mean>=50)&
                                   (mt_snv_p.info.MQRankSum>=-1.7))
mt_snv_het_HQ = mt_snv_het_HQ.annotate_entries(AB=hl.cond(mt_snv_het_HQ.GT.is_het(),
                                                          hl.min(mt_snv_het_HQ.AD.map(lambda x: x/mt_snv_het_HQ.DP)),0))
mt_snv_het_HQ = mt_snv_het_HQ.filter_entries((mt_snv_het_HQ.GT.is_het())&
                                             (mt_snv_het_HQ.GQ>=99)&
                                             (mt_snv_het_HQ.DP>=10)&
                                             (mt_snv_het_HQ.AB>=0.24)&
                                             (mt_snv_het_HQ.AB<=0.76))
mt_snv_het_HQ = hl.sample_qc(mt_snv_het_HQ)

---

### 4. Export data + CSQ annotation


(CSQ 전체)

In [25]:
def CSQ(table):
    t = table
    t = t.annotate(v1 = hl.tuple([t.locus.contig.replace("chr", ""),hl.str(t.locus.position)]),
                   v2 = hl.tuple([t.alleles[0],t.alleles[1]]))
    t = t.transmute(variant = hl.delimit(hl.array([t.v1[0],t.v1[1],t.v2[0],t.v2[1]]), ":"))
    t = t.annotate(CSQ= t.info.CSQ).explode('CSQ')
    t = t.transmute(csq = t.CSQ.split('\|')).explode('csq')
    return t

In [26]:
tb = CSQ(tb)

In [27]:
tb.export("/titan2/UDP_SV/df_csq_single_hom_12.tsv")

2020-07-17 17:26:51 Hail: INFO: Coerced sorted dataset
2020-07-17 17:26:51 Hail: INFO: Coerced sorted dataset
2020-07-17 17:26:51 Hail: INFO: Coerced sorted dataset
2020-07-17 17:27:01 Hail: INFO: merging 2 files totalling 956.4M...
2020-07-17 17:27:02 Hail: INFO: while writing:
    /titan2/UDP_SV/df_csq_single_hom_12.tsv
  merge time: 1.718s
